In [1]:
# default inputs
import tellurium as te
import matplotlib.pyplot as plt
%config Completer.use_jedi = False
showOriginalModelString = True

In [2]:
if showOriginalModelString:
    model = te.loadSBMLModel("Mg_M.xml")
    modelStr = model.getAntimony()
#     print(modelStr)
    

In [3]:
def run(model,selections, duration):
    model.integrator.absolute_tolerance = 1e-9
    model.integrator.relatice_tolerance = 1e-9
    results = model.simulate(start = 0, end = duration,steps = duration,
                             selections = selections)
    return results
def modify(modelStr,replacements):
    for key,value in replacements.items():
        modelStr = modelStr.replace(key,value)
    return modelStr
def merge(modelStr,sub_model):
    rr = modelStr.split('\nend\n\npad_mac')
    combined = rr[0]+sub_model+'\nend\n\npad_mac is "pad mac"\n'
    return combined

In [4]:
# replacements = {
#     '$ikk_prod => IKK; k191*ikk_prod':'NEMO_IKK + Mg => Mg_NEMO + IKK; k302*Mg*NEMO_IKK - k303*IKK - k304*Mg_NEMO',
#     'IKK + pTAK1 => pIKK + pTAK1; k189*IKK*pTAK1^2': 'IKK + pTAK1 => pIKK + pTAK1; k305*IKK*pTAK1^2',
#     'IKK + O2_aPHD => deg + O2_aPHD;  k192*IKK*O2_aPHD^2': 'IKK + O2_aPHD => deg + O2_aPHD; k306*IKK*O2_aPHD^2',
#     'pIKK => IKK; k190*pIKK': 'pIKK => IKK; k307*pIKK'
# }
replacements = {
#     '$ikk_prod => IKK; k191*ikk_prod':'NEMO_IKK + Mg => Mg_NEMO + IKK; k302*Mg*NEMO_IKK- k303*IKK - k304*Mg_NEMO'
}
extra_additions = """
    #// Mg initial condition and diffusion
    Mg_e = 0.8*Mg_copy;
    Mg = 0.8*Mg_copy ; # internal/physiological Mg
    $Mg_e -> Mg; k301*(Mg_e - Mg) ; # Mg diffuses from extra- to intracellular space through345 TRPM7  
    Mg_copy = 1000;k301 = 0;
    
    #// params related to the NEMO_IKK + Mg interaction
    #Mg_NEMO = 1000;
    #k302=0.01; k303 = 0.01; k304=0.01;
    
    #// NEMO_IKK production and degrdation
    #$NEMO_IKK_prod => NEMO_IKK; k308-k310*NEMO_IKK; #production
   # Mg_NEMO -> deg; k309*Mg_NEMO; # degradation
   # NEMO_IKK_prod = 1; NEMO_IKK = 1000;k310=100;k309=.1;k308=100;
    
    #// TRMP production
    #$TRPM_prod -> TRPM ; k311*Mg^n300 - k312*TRPM;
    #TRPM -> deg; k313*TRPM
    #TRPM_prod = 1; TRPM = 1000; k311 = .1; k312 = .1; k313 = .1;n300=1
    
    #// activation of M7CKs
    #TRPM7 -> M7CKs; k314*Mg*TRPM7 - k315*M7CKs;
    #M7CKs = 1000; k314 = 100; k315 = 100;
    
    #// nuclear translocation of M7CKs and vice versa
    #M7CKs -> M7CKs_n; k316*M7CKs*Mg  - k317*M7CKs_n
   # M7CKs_n -> M7CKs ; k318*M7CKs_n - k319*M7CKs
    #M7CKs_n= 100 ; k316=100 ; k317=100; k318=100; k319=100;
    
    #// activation and phosphorylation of H3S10
    #$H3S10_prod -> H3S10; k320*M7CKs_n - k321*H3S10 ;
    #H3S10 -> deg; k322*H3S10;
    #H3S10 -> pH3S10; k323*M7CKs_n*H3S10-k324*pH3S10
    #H3S10_prod = 1; H3S10 = 100; pH3S10 = 1000; k320=100; k321 = 100; k322 = 100; k323 = 100; k324 = 100; 
    
    #// IL8 production, extracellular and intracellular transportation
    #$IL8_prod -> IL8; k325*pH3S10 - k326*IL8;
    #IL8 -> deg; k327*IL8;
    #IL8 -> IL8_e; k328*IL8 - k329*IL8_e;
    #IL8_e -> IL8; k330*IL8_e - k331*IL8;
    #IL8 = 100; k325 = 100; k326 = 100; k327 = 100; k328 = 100; k329 = 100; k330 = 100; k331 = 100;
    
"""
originalModel = te.loadSBMLModel("Zhao_2021.xml")
originalModelStr = originalModel.getAntimony ()
modified_model = modify(originalModelStr,replacements)
combined = merge(modified_model,extra_additions)
combined = te.loada(combined)
combined.exportToSBML('Mg_M.xml')


In [5]:
if False:
    def initial_conditions(model,keys,values):
        for i in range(len(keys)):
            free_param_name = keys[i]
            model[free_param_name] = values[i]

    def run(model,targets, duration):
        model.integrator.absolute_tolerance = 1e-9
        model.integrator.relatice_tolerance = 1e-9
        results = model.simulate(start = 0, end = duration,steps = duration,
                                 selections = targets)
        return results

    targets = ['TRPM']
    model = combined
    model.reset()
    initial_conditions(model,['k311','k312'],[.001,1])
    # initial_conditions(model,['k301','Mg_e_mM','Mg_copy'],[.1,.8,1000])

    results = run(model,targets,72*60)
    for tag in targets:
        fig = plt.figure()
        plt.plot(results[tag],label = 'TRPM',linestyle = '--')
        plt.legend()
        plt.title(tag)

In [6]:
# test = """
#     NEMO_IKK + Mg => Mg_NEMO + IKK; k302*Mg*NEMO_IKK- k303*IKK - k304*Mg_NEMO;
#     #NEMO_IKK + Mg => Mg_NEMO + IKK; k302*Mg*NEMO_IKK;
#     NEMO_IKK = 1000; Mg=1000; Mg_NEMO= 1000; IKK = 800; 
#     k302=.00011;k303=.001;k304=.001;
# """
# testModel = te.loada(test)
# targets = ['Mg']
# results = testModel.simulate(0,100,100,selections = ['TIME']+targets)
# for tag in targets:
#     fig = plt.figure()
#     plt.plot(results[tag],label = 'Mg',linestyle = '--')
#     plt.legend()
#     plt.title(tag)